# **재정정보 AI 검색 알고리즘 경진대회 - RAG, **
> **RAG, **

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

---

💡 **NOTE**: 이 예제에서 사용한 모델 및 벡터 DB 

1. LLM : mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1
2. Embed Model : intfloat/multilingual-e5-large
3. Vector DB : FAISS

---

# Install

In [ ]:
# 모델 가속화 및 메모리 관리
%pip install accelerate
%pip install -i https://pypi.org/simple/ bitsandbytes

# Transformer 기반 모델과 데이터셋 관련 라이브러리
%pip install transformers[torch] -U
%pip install datasets

# Llama-Index
%pip install llama-index
%pip install llama-index-llms-huggingface
%pip install llama-index-vector-stores-faiss
%pip install llama-index-postprocessor-cohere-rerank
%pip install llama-index-readers-file pymupdf

# 벡터 데이터베이스 관련 라이브러리
%pip install faiss-cpu

# 데이터
%pip install umap-learn
%pip install scikit-learn
%pip install tiktoken

%load_ext autoreload
%autoreload 2

# 라이브러리 불러오기

In [ ]:
import os
import json
from tqdm import tqdm
import unicodedata
import numpy as np
import pandas as pd
import nest_asyncio
import logging
import sys
import torch
import pymupdf4llm
from typing import Dict, List, Optional, Tuple


from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Gemma2ForCausalLM,
    TextStreamer,
)
from accelerate import Accelerator

# Llama-Index
from llama_index.core import (
    VectorStoreIndex, 
    SimpleDirectoryReader, 
    StorageContext,
    Document, 
    Settings,
    PromptTemplate,
    ChatPromptTemplate,
    get_response_synthesizer,
    DocumentSummaryIndex,
    SummaryIndex
)
from llama_index.core.response.notebook_utils import display_response

from llama_index.llms import HuggingFaceLLM
# from llama_index.core.response_synthesizers import TreeSummarize, Refine
# from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores import FAISSVectorStore


from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

# 초기 설정

In [ ]:
# 객체 생성시 llm_name, embed_model_name 인가

class Config:
    def __init__(self, llm_name: str = "meta-llama/Meta-Llama-3.1-8B-Instruct", embed_model_name: str = "intfloat/multilingual-e5-base") -> None:        
        
        # 모델별 설정 딕셔너리
        self.model_configs = {
            "meta-llama/Meta-Llama-3.1-8B-Instruct": {
                "quantization_config": None,
                "torch_dtype": "auto",
                "max_token": 256,
            },
            "rtzr/ko-gemma-2-9b-it": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": "auto",
                "max_token": 450,
            },
            "jjjguz/Llama-3.1-Korean-8B-Instruct-v1": {
                "quantization_config": None,
                "torch_dtype": "auto",
                "max_token": 256,
            },
            "mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": "auto",
                "max_token": 450,
            },
        }
        
        # LLM 모델 설정
        self.llm_name = llm_name
        self.llm_model_config = self.model_configs[self.llm_name]
        self.llm_obj = self.setup_llm_pipeline()
        
        # 임베딩 모델 설정
        self.embed_model_name = embed_model_name 
        self.embed_model_obj = self.setup_embeddings()
        
        # 벡터 DB 설정
        self.base_directory = "open/"
        self.train_csv_path = os.path.join(self.base_directory, "train.csv")
        self.test_csv_path = os.path.join(self.base_directory, "test.csv")
        self.chunk_size = 512
        self.chunk_overlap = 32
                
        # 제출 및 평가 설정
        self.is_submit = True
        self.eval_sum_mode = False
        
        # 출력 디렉토리 및 파일 설정
        self.output_dir = "test_results"
        self.output_csv_file = (
            f"{self.llm_name.split('/')[1]}_{self.embed_model_name.split('/')[1]}_"
            f"pdf_loader_chks{self.chunk_size}_chkovp{self.chunk_overlap}_submission.csv"
        )
        os.makedirs(self.output_dir, exist_ok=True)
    
    def to_json(self):
        return json.dumps(self.__dict__)    
    
    def get_quantization_config(self):
        """4-bit 양자화 설정을 반환하는 함수"""
        return BitsAndBytesConfig(
            load_in_4bit=True,  # 4-bit 양자화
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16  # 연산에 사용할 데이터 타입
        )
        
    def setup_embeddings(self):
        """ 임베딩 모델 설정 """
        embed_id = self.embed_model_name
        
        model_kwargs = {'device': 'cuda'}
        encode_kwargs = {'normalize_embeddings': True}
        embd = HuggingFaceEmbeddings(
            model_name=embed_id,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
        )
        
        store = LocalFileStore("./cache/")
        
        # Cache Embedding 사용
        cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
            underlying_embeddings=embd, 
            document_embedding_cache=store, 
            namespace=embed_id
        )
        
        return cached_embeddings
        
    def setup_llm_pipeline(self):
        """LLM 설정 및 파이프라인 구성"""
        model_id = self.llm_name
        
        # 토크나이저 로드 및 설정
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        tokenizer.use_default_system_prompt = False
        
        # 모델 로드 및 양자화 설정 적용
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            quantization_config=self.llm_model_config["quantization_config"],
            trust_remote_code=True
        )
        
        # 모델을 여러 GPU에 할당
        accelerator = Accelerator()
        model = accelerator.prepare(model)
        
        print(f"#### [ model ] ####\n{model}\n###################")
        
        # 스트리머를 설정하여 토큰이 생성될 때마다 출력
        streamer = TextStreamer(tokenizer)

        # HuggingFacePipeline 객체 생성
        text_generation_pipeline = pipeline(
            model=model,
            tokenizer=tokenizer,
            task="text-generation",
            return_full_text=False,
            max_new_tokens=450,
            streamer=streamer
        )

        hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

        return hf

In [ ]:
# 비동기 처리
nest_asyncio.apply()

# 로깅 설정
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

# GPU 설정
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 데이터셋 전처리 및 벡터 DB 구축

https://docs.llamaindex.ai/en/stable/examples/retrievers/ensemble_retrieval/ 

참고

In [ ]:
import re

# 초기화시 config 객체, raptor 객체 넣기
class PDFProcessor:
    def __init__(self, config, raptor, embed_obj):
        self.config = config
        self.raptor = raptor
        self.embed_obj  = embed_obj
        logger.info("PDFProcessor 초기화 완료.")
                
    def normalize_path(self, path):
        """ Path 유니코드 정규화 """
        normalized_path = unicodedata.normalize('NFD', path)
        logger.debug(f"정규화된 경로: {normalized_path}")
        return normalized_path
    
    def process_pdf(self, file_path) -> List[str]:
        """ PDF 파일 로드, 텍스트 추출 """
        logger.info(f"PDF 처리 중: {file_path}")
        
        loader = PyMuPDFReader()
        docs0 = loader.load(file_path=Path(file_path))
        doc_text = "\n\n".join([d.get_content() for d in docs0])
        docs = [Document(text=doc_text)]
        
        # docs_md_content = pymupdf4llm.to_markdown(file_path)

        # # 마침표 뒤 및 "----" 전후의 줄바꿈을 제외한 모든 줄바꿈을 제거
        # processed_text = re.sub(r'(?<!\.)(?<!-----)(\n|\r\n)(?!-----)', ' ', docs_md_content)
        # logger.debug("불필요한 줄바꿈 제거 완료.")
        
        # chunk_size_tok = 2000

        # text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        #     chunk_size=chunk_size_tok, chunk_overlap=0
        # )
        # splits = text_splitter.split_text(processed_text)
        # logger.info(f"텍스트를 {len(splits)}개의 청크로 분할 완료.")
        
        return splits
    
    def process_pdfs_from_dataframe(self, df):
        """ PDF 데이터셋으로부터 벡터 DB 구축 """
        pdf_databases = {}
        unique_paths = df['Source_path'].unique()

        for path in tqdm(unique_paths, desc="Processing PDFs"):
            # 경로 정규화 및 절대 경로 생성
            normalized_path = self.normalize_path(path)
            full_path = os.path.normpath(os.path.join(self.config.base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

            # pdf -> 벡터 DB 구축
            pdf_title = os.path.splitext(os.path.basename(full_path))[0]
            db_index_path = f"./RAPTOR_{pdf_title}.faiss"
            
            
            if os.path.exists(db_index_path):
                logging.info(f"기존 벡터 DB 로드 중: {db_index_path}")
                vector_store = FAISS.load_local(db_index_path, embeddings=self.embed_obj, allow_dangerous_deserialization=True)
            else:
                print(f"PDF -> 벡터 DB 구축 중 : [{pdf_title}]...")
            
                leaf_texts = self.process_pdf(full_path) # list[str]

                # RAPTOR 트리 구축
                raptor_results = self.raptor.recursive_embed_cluster_summarize(
                    leaf_texts, level=1, n_levels=3
                )
                logger.info(f"RAPTOR 트리 구축 완료: {pdf_title}.")
            
                # 각 레벨의 요약을 추출하여 all_texts에 추가
                all_texts = leaf_texts.copy()
                for level in sorted(raptor_results.keys()):
                    summaries = raptor_results[level][1]["summaries"].tolist()  # 현재 레벨의 DataFrame에서 요약을 추출합니다.
                    all_texts.extend(summaries)                                 # 현재 레벨의 요약을 all_texts에 추가합니다.
                logger.info(f"{pdf_title}에 대해 요약 추가 완료.")
            
                # FAISS 벡터 데이터베이스 구축
                vector_store = FAISS.from_texts(texts=all_texts, embedding=self.embed_obj)
                vector_store.save_local(db_index_path)
                logger.info(f"벡터 데이터베이스 저장 경로: {db_index_path}")
            
            retriever = vector_store.as_retriever()
            
            pdf_databases[pdf_title] = {
                'db': vector_store,
                'retriever' : retriever,
                'index_path': db_index_path
            }
            logger.info(f"PDF 처리 완료: {pdf_title}.")
        
        logger.info("모든 PDF 처리 완료.")
        return pdf_databases
    

# QA 생성 모듈

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

class QAGenerator:
    def __init__(self, config, llm):
        self.config = config
        self.llm = llm
    
    def normalize_string(self, s):
        """유니코드 정규화"""
        return unicodedata.normalize('NFC', s)
    
    def format_docs(self, docs):
        # 문서의 페이지 내용을 이어붙여 반환합니다.
        return "\n\n".join(doc.page_content for doc in docs)
    
    def clean_answer(self, answer):
        # 다양한 불필요한 텍스트 패턴을 제거
        patterns_to_remove = [
            r"^\s*###? Response:\s*",
            r"^\s*Response:\s*",
            r"^\s*AI:\s*",
            r"^\s*Quad:\s*",
            r"^\s*\|\-\s*\n?\s*\n?\s*###? Response:\s*",
            r"^\s*\|\-\s*\n?\s*\n?\s*Response:\s*",
            r"^\s*\|\-\s*\n?\s*\n?\s*AI:\s*",
            r"^\s*\|\-\s*\n?\s*\n?\s*Quad:\s*",
            r"<eos>\s*$"
        ]
        for pattern in patterns_to_remove:
            answer = re.sub(pattern, "", answer)
        return answer.strip()
        
    def generate_answers(self, df, pdf_databases):
        """DataFrame의 각 질문에 대해 답변을 생성"""
        
        results = []
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
            source = self.normalize_string(row['Source'])
            query = row['Question']

            # 정규화된 키로 데이터베이스 검색
            normalized_keys = {self.normalize_string(k): v for k, v in pdf_databases.items()}
            retriever = normalized_keys[source]['retriever']
            
            # 프롬프트 엔지니어링
            system_prompt_template = """
            당신은 중앙정부 재정 정책 전문가입니다. 주어진 재정 정보를 바탕으로, 명확하고 정확한 답변을 제시하세요.
            답변을 생성할 때에는 "Response:", "AI:" 등과 같은 불필요한 텍스트를 포함하지 말고, 핵심 정보만 명확하게 전달하세요.
            
            주어진 질문에 대해 다음 단계에 따라 답변하세요:
            1. 질문을 이해하고 필요한 재정 개념을 설명합니다.
            2. 질문을 단계별로 분석하여 논리적인 답변을 구성합니다.
            3. 관련 개념 간의 관계를 분석하여 맥락을 명확히 합니다.
            
            답변은 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 파악하여 답변하세요.
            단, 주어와 서술어를 적절히 사용하여 온전한 문장을 완성시켜 최대한 자연스럽게 답변해야 합니다.
            
            다음 정보를 바탕으로 질문에 답하세요 :
            {context}            
            """

            # 프롬프트 생성
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", system_prompt_template),
                    ("human", "{input}"),
                ]
            )

            qa_chain = create_stuff_documents_chain(llm=self.llm, prompt=prompt)
            rag_chain = create_retrieval_chain(retriever, qa_chain)
            
            # 답변 추론
            print(f"Question: {query}")
            result = rag_chain.invoke({"input": query})
            print(f"Answer: {result}\n")

            # 결과 저장
            results.append({
                "Source": row['Source'],
                "Source_path": row['Source_path'],
                "Question": query,
                "Answer": result
            })

        return results

---

# 최종 통합, 실행 및 저장

In [ ]:
if __name__ == "__main__":
    config = Config(
        llm_name         = "mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1", 
        embed_model_name = "intfloat/multilingual-e5-large"
    )
    
    # RaptorClustering 객체 초기화
    raptor = RaptorClustering(
        embed_obj=config.embed_model_obj,
        llm_obj=config.llm_obj
    )

    # PDFProcessor 객체 초기화 및 PDF 데이터셋으로부터 벡터 DB 구축
    pdf_processor = PDFProcessor(config=config, raptor=raptor, embed_obj=config.embed_model_obj)
    
    # 데이터를 로드하고 벡터 DB를 구축
    test_df = pd.read_csv(config.test_csv_path)
    pdf_databases = pdf_processor.process_pdfs_from_dataframe(test_df)
    
    # 질문에 대한 답변 생성
    qa_generator = QAGenerator(config=config, llm=config.llm_obj)
    results = qa_generator.generate_answers(test_df, pdf_databases)

    # 제출 샘플 파일 로드 및 답변 추가
    submit_df = pd.read_csv(f"./open/sample_submission.csv")
    submit_df['Answer'] = [
        qa_generator.clean_answer(item['Answer']['answer']) for item in results
    ]
    submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")

    # 결과를 CSV 파일로 저장
    submit_df.to_csv(os.path.join(config.output_dir, config.output_csv_file), encoding='UTF-8-sig', index=False)